# T0. PREPARE POINT CLOUDS

Import and prepare CVPR point clouds for t1-t10.
To run these scripts, create a python 3.10 environment & install geomapi (numpy, opend3d, ifcopenshell, trimesh, ...)

## LIBRARIES

In [1]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import time
import json
from scipy.spatial.transform import Rotation   
import copy
from datetime import datetime

import geomapi
from geomapi.nodes import *
from geomapi import utils as ut
from geomapi.utils import geometryutils as gmu
from geomapi.tools import progresstools as pt

import geomapi.tools as tl

import context
import utils as utl
import utils.t0_utils as t0


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [4]:
path=Path(os.getcwd()).parents[2]/'data'
input_folder=path / 't0'/'train'
class_file=path/'_classes.json'
output_folder= path/'t0'/'results'
output_folder.mkdir(parents=True, exist_ok=True)
# graphPath=str(path/f'objectGraph.ttl')

distance_threshold=0.1
resolution=0.03

import classes

In [5]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


## PROCESSING

In [6]:
files=utl.get_list_of_files(input_folder,'.laz')

#iterate over files
for f in files: 
    start_time = time.time()
    #read pcd
    print(f)   
    
    # #create pcdNode
    las=laspy.read(f)
    name='_'.join(f.split('/')[-1].split('.')[0].split('_')[0:-1])
    # print(name)
    # pcdNode=PointCloudNode(resource=gmu.las_to_pcd(las),
    #                                 name=name,
    #                                 color=ut.random_color()) 
    
    #process jsons
    json_files=[file for file in utl.get_list_of_files(input_folder,'.json') if name in ut.get_filename(file)]   
    objects_dict = {}  
    objectNodes=[]
    for file in json_files:
        # Load the JSON data into Python objects
        t0.parse_json(file,objects_dict)

    # Print counts
    print(f'len columns: {sum(1 for item in objects_dict.values() if item["type"] == "columns")}')
    print(f'len doors: {sum(1 for item in objects_dict.values() if item["type"] == "doors")}')
    print(f'len walls: {sum(1 for item in objects_dict.values() if item["type"] == "walls")}')

    # #convert json to object nodes
    objectNodes=t0.create_object_nodes(objects_dict,class_dict)

    # #map to point clouds
    # class_scalar,object_scalar=t0.process_point_cloud(pcdNode,
    #                                                   objectNodes,
    #                                                   distance_threshold=distance_threshold,
    #                                                   resolution=resolution)
    
    # #alter door nodes
    # doorNodes=[n for n in objectNodes n.class_id==4]
    # points=o3d.utility.Vector3dVector(las.xyz)
    # for i,d in enumerate(doorNodes):
    #     box=d.resource.get_oriented_bounding_box()
    #     box=gmu.expand_box(box,0.05,0.05,0.05)
    #     indices=box.get_point_indices_within_bounding_box(points)
    #     class_scalar[indices]=d.class_id
    #     object_scalar[indices]=i+400       
    
    # gmu.las_add_extra_dimensions(las,(class_scalar,object_scalar),['classes','objects'],['uint8','uint16'])
    
    # las.write(output_folder/f'{pcdNode.name}.laz')
    # print(f'{pcdNode.name}_pcd.laz')    

    #export graphs
    tl.nodes_to_graph(objectNodes,
                    graphPath=str(output_folder/f'{name}.ttl'),
                    save=True)
    
    #export objects
    t0.write_obj_with_submeshes(output_folder/f'{name}_objects.obj', [n.resource for n in objectNodes], [n.name for n in objectNodes])
    print(f'{name} written!')
    
    print(f'Elapsed time: {time.time() - start_time}')

c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t0/train/LAZ_train/05_MedOffice_01_F2_s0p01m.laz
Data read from file: 05_MedOffice_01_F2_columns
Data read from file: 05_MedOffice_01_F2_doors
Data read from file: 05_MedOffice_01_F2_walls
len columns: 25
len doors: 62
len walls: 161
05_MedOffice_01_F2 written!
Elapsed time: 2.001203775405884
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t0/train/LAZ_train/06_MedOffice_02_B1_s0p01m.laz
Data read from file: 06_MedOffice_02_B1_columns
Data read from file: 06_MedOffice_02_B1_doors
Data read from file: 06_MedOffice_02_B1_walls
len columns: 71
len doors: 11
len walls: 62
06_MedOffice_02_B1 written!
Elapsed time: 6.488832473754883
c:/Users/u0094523/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/t0/train/LAZ_train/06_MedOffice_02_F1_s0p01m.laz
Data read from file: 06_MedOffice_02_F1_columns
Data read from file: 06_MedOffice_02_F1_doors
Data read from file: 06_MedOffice

In [7]:
# joined_geometries=gmu.join_geometries([d.resource for d in doorNodes if d.derivedFrom in f])
# # lines=gmu.join_geometries([n.line.paint_uniform_color(n.color) for n in objectNodes if n.derivedFrom==pcdNode.name 
# #                            if getattr(n,'line',None) is not None])

# o3d.visualization.draw_geometries([joined_geometries])

In [8]:
# o=0
# walls=gmu.join_geometries([n.resource.paint_uniform_color([1,0,0]) for n in objectNodes if n.derivedFrom==pcdNodes[o].name
#                            if n.class_name in ['walls']])
# columns=gmu.join_geometries([n.resource.paint_uniform_color([0,1,0]) for n in objectNodes if n.derivedFrom==pcdNodes[o].name
#                              if n.class_name in ['columns']])
# doors=gmu.join_geometries([n.resource.paint_uniform_color([0,0,1]) for n in objectNodes if n.derivedFrom==pcdNodes[o].name
#                             if n.class_name in ['doors']])
# joined_geometries=gmu.join_geometries([walls,columns,doors])
# print(joined_geometries)

# lines=gmu.join_geometries([n.line.paint_uniform_color(n.color) for n in objectNodes if n.derivedFrom==pcdNodes[o].name 
#                            if getattr(n,'line',None) is not None])

# o3d.visualization.draw_geometries([pcdNodes[o].resource] + [joined_geometries,lines])